# Distributed random-search hyper-parameter optimization of the Keras RPV classifier

In [ ]:
# System imports
from __future__ import print_function
from __future__ import division
from __future__ import absolute_import
import os

# External imports
import ipyparallel as ipp
import numpy as np
import keras
import matplotlib.pyplot as plt

# Local imports
from rpv import load_file

## Connect to cluster

In [ ]:
%%bash
squeue -u sfarrell

In [ ]:
# Cluster ID taken from job ID above
job_id = 24194075
cluster_id = 'cori_{}'.format(job_id)

# Use default profile
c = ipp.Client(timeout=60, cluster_id=cluster_id)
print('Worker IDs:', c.ids)

## Configure the hyper-parameter search tasks

In [ ]:
# Data config
n_train = 64000 #412416
n_valid = 32000 #137471
n_test = 32000 #137471
input_dir = '/global/cscratch1/sd/sfarrell/atlas-rpv-images'

In [ ]:
# Temporarily making things reproducible for development
np.random.seed(0)

# Define the hyper-parameter search points
n_hpo_trials = 32
h1 = np.random.choice([4, 8, 16, 32, 64], size=n_hpo_trials)
h2 = np.random.choice([4, 8, 16, 32, 64], size=n_hpo_trials)
h3 = np.random.choice([8, 16, 32, 64, 128], size=n_hpo_trials)
conv_sizes = np.stack([h1, h2, h3], axis=1)
fc_sizes = np.random.choice([32, 64, 128, 256], size=(n_hpo_trials, 1))
lr = np.random.choice([0.0001, 0.001, 0.01], size=n_hpo_trials)
dropout = np.random.rand(n_hpo_trials)
optimizer = np.random.choice(['Adadelta', 'Adam', 'Nadam'], size=n_hpo_trials)

# Training config
batch_size = 64
n_epochs = 16
checkpoint_dir = os.path.join(os.environ['SCRATCH'],
                              'cori-interactive-dl/rpv_hpo_%i' % job_id)
os.makedirs(checkpoint_dir, exist_ok=True)

## Run the hyper-parameter search

In [ ]:
def build_and_train(input_dir, n_train, n_valid,
                    conv_sizes, fc_sizes, dropout, optimizer, lr,
                    batch_size, n_epochs, checkpoint_file=None, verbose=2):
    """Run training for one set of hyper-parameters"""
    import os
    import keras
    from rpv import build_model, train_model, load_file
    from mlextras import configure_session
    print('Hyperparameters: conv %s fc %s dropout %.3f opt %s, lr %.4f' %
          (conv_sizes, fc_sizes, dropout, optimizer, lr))
    # Load the dataset
    train_input, train_labels, train_weights = load_file(os.path.join(input_dir, 'train.h5'), n_train)
    valid_input, valid_labels, valid_weights = load_file(os.path.join(input_dir, 'val.h5'), n_valid)
    print('train shape:', train_input.shape, 'Mean label:', train_labels.mean())
    print('valid shape:', valid_input.shape, 'Mean label:', valid_labels.mean())
    # Thread settings
    keras.backend.set_session(configure_session())
    # Build the model
    model = build_model(train_input.shape[1:],
                        conv_sizes=conv_sizes, fc_sizes=fc_sizes,
                        dropout=dropout, optimizer=optimizer, lr=lr)
    # Train the model
    history = train_model(model, train_input=train_input, train_labels=train_labels,
                          valid_input=valid_input, valid_labels=valid_labels,
                          batch_size=batch_size, n_epochs=n_epochs,
                          checkpoint_file=checkpoint_file, verbose=verbose)
    return history.history

In [ ]:
# Load-balanced view
lv = c.load_balanced_view()

# Loop over hyper-parameter sets
results = []
for ihp in range(n_hpo_trials):
    print('Hyperparameter trial %i conv %s fc %s dropout %.4f opt %s, lr %.4f' %
          (ihp, conv_sizes[ihp], fc_sizes[ihp], dropout[ihp], optimizer[ihp], lr[ihp]))
    checkpoint_file = os.path.join(checkpoint_dir, 'model_%i.h5' % ihp)
    result = lv.apply(build_and_train,
                      input_dir, n_train, n_valid,
                      conv_sizes=conv_sizes[ihp], fc_sizes=fc_sizes[ihp],
                      dropout=dropout[ihp], optimizer=optimizer[ihp], lr=lr[ihp],
                      batch_size=batch_size, n_epochs=n_epochs,
                      checkpoint_file=checkpoint_file)
    results.append(result)

In [ ]:
print('Tasks completed: %i / %i' % (np.sum([ar.ready() for ar in results]), len(results)))

In [ ]:
# Print standard out for one of the runs
ar = results[1]
print('STDOUT')
print(ar.stdout)
print('STDERR')
print(ar.stderr)

In [ ]:
# Pull the results from all of the runs that have finished
histories = [ar.get() for ar in results]

In [ ]:
times = [(ar.completed-ar.started).total_seconds()/60. for ar in results]

In [ ]:
%matplotlib inline

In [ ]:
plt.figure()
plt.hist(times)
plt.xlabel('Training time [min]');

## Model selection and evaluation

In [ ]:
def draw_history(h):
    plt.figure(figsize=(9,4))
    # Loss
    plt.subplot(121)
    plt.plot(h['loss'], label='Training')
    plt.plot(h['val_loss'], label='Validation')
    plt.xlim(xmin=0, xmax=len(h['loss'])-1)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(loc=0)
    # Accuracy
    plt.subplot(122)
    plt.plot(h['acc'], label='Training')
    plt.plot(h['val_acc'], label='Validation')
    plt.xlim(xmin=0, xmax=len(h['loss'])-1)
    plt.ylim((0, 1))
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(loc=0)
    plt.tight_layout()

In [ ]:
last_scores = np.array([h['val_acc'][-1] for h in histories])
best_scores = np.array([max(h['val_acc']) for h in histories])

In [ ]:
# Find the best in terms of final validation set accuracy
i = best_scores.argmax()
print('Hyperparameters: trial %i conv %s fc %s dropout %.3f opt %s, lr %.3f' %
      (i, conv_sizes[i], fc_sizes[i], dropout[i], optimizer[i], lr[i]))
print('  Last validation accuracy %.4f' % last_scores[i])
print('  Best validation accuracy %.4f' % best_scores[i])
draw_history(histories[i])

In [ ]:
# Find the worst in terms of final validation set accuracy
i = best_scores.argmin()

print('Hyperparameters: trial %i conv %s fc %s dropout %.3f opt %s, lr %.3f' %
      (i, conv_sizes[i], fc_sizes[i], dropout[i], optimizer[i], lr[i]))
print('  Last validation accuracy %.4f' % last_scores[i])
print('  Best validation accuracy %.4f' % best_scores[i])
draw_history(histories[i])

### Looking for trends in the best hyper-parameter sets

In [ ]:
plt.figure()
plt.hist(best_scores, bins=20, range=(0.5, 1))
plt.xlabel('Validation accuracy');

In [ ]:
for i in best_scores.argsort()[::-1][:5]:
    print('Hyperparameters: trial %i conv %s fc %s dropout %.3f opt %s, lr %.3f' %
          (i, conv_sizes[i], fc_sizes[i], dropout[i], optimizer[i], lr[i]))
    print('  Best validation accuracy %.4f' % best_scores[i])

### Test set evaluation

Here we load the best selected model and evaluate final performance on the test set.

In [ ]:
from sklearn import metrics

def summarize_metrics(labels, outputs, threshold=0.5, weights=None):
    preds = outputs > threshold
    #print('Metrics summaries with threshold of %.3f' % threshold)
    print('Accuracy:   %.4f' % metrics.accuracy_score(labels, preds, sample_weight=weights))
    print('Purity:     %.4f' % metrics.precision_score(labels, preds, sample_weight=weights))
    print('Efficiency: %.4f' % metrics.recall_score(labels, preds, sample_weight=weights))

def draw_roc(labels, outputs, weights=None, ax=None):
    fpr, tpr, _ = metrics.roc_curve(labels, outputs, sample_weight=weights)
    auc = metrics.roc_auc_score(labels, outputs, sample_weight=weights)
    if ax is None:
        fig, ax = plt.subplots()
    ax.plot(fpr, tpr, label='CNN, AUC=%.3f' % auc)
    ax.plot([0, 1], [0, 1], '--', label='Random')
    ax.set_xlabel('False positive rate')
    ax.set_ylabel('True positive rate')
    ax.legend(loc=0)

In [ ]:
test_input, test_labels, test_weights = load_file(os.path.join(input_dir, 'test.h5'), n_test)

In [ ]:
i = best_scores.argmax()
model_file = os.path.join(checkpoint_dir, 'model_%i.h5' % i)
model = keras.models.load_model(model_file)

In [ ]:
test_output = model.predict(test_input)
test_output = test_output.squeeze(-1)

In [ ]:
# Unweighted results
print('Unweighted metrics')
summarize_metrics(test_labels, test_output)

# Weighted results
print('Weighted metrics')
summarize_metrics(test_labels, test_output, weights=test_weights)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(9,4))
draw_roc(test_labels, test_output, ax=axs[0])
draw_roc(test_labels, test_output, ax=axs[1], weights=test_weights)
axs[0].set_xlim([0, 0.001])
axs[0].set_title('Unweighted')
axs[1].set_xlim([0, 0.001])
axs[1].set_title('Weighted');